In [3]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import numpy as np
import os
import time
#from vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.applications.imagenet_utils import decode_predictions
from keras.layers import Dense, Activation, Flatten
from keras.layers import merge, Input
from keras.models import Model
from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
#import tf.kerasTasm
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
# Loading the training data
from PIL import ImageOps

# Define data path
# Define data path
data_path = r'/floyd/input/images/Number'
data_dir_list = os.listdir(data_path)
data_dir_list



# In[2]:

desired_size = 224


# In[3]i:


img_data_list=[]
labels = np.array([])
for dataset in data_dir_list:
    img_list=os.listdir(data_path+'/'+ dataset)
    print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
    for img in img_list:
        img_path = data_path + '/'+ dataset + '/'+ img
        img = image.load_img(img_path, target_size=(desired_size, desired_size))
        # img = img.resize((desired_size, desired_size))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        # x = x/255 destroys accuracy, type issues?
        img_data_list.append(x)
        
    label = np.full((len(img_list),), int(dataset),dtype='float64')
    labels = np.append(labels, label)

img_data = np.array(img_data_list)
img_data=np.rollaxis(img_data,1,0)
img_data=img_data[0]


# convert class labels to on-hot encoding
# Y = np_utils.to_categorical(labels, num_classes=1)
Y = labels

#Shuffle the dataset
x,y = shuffle(img_data,Y, random_state=2) # TAKES TIME 
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=2) # TAKES TIME
X_test, X_val, y_test, y_val = train_test_split( X_test, y_test, test_size=0.3, random_state=2) # TAKES TIME
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)


# In[19]:


#Training the classifier alone

image_input = Input(shape=(desired_size, desired_size, 3))

model = VGG19(input_tensor=image_input, include_top=True, weights='imagenet')
# (include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)
model.summary()
last_layer = model.get_layer('fc2').output
#x= Flatten(name='flatten')(last_layer)
out = Dense(1, activation='sigmoid', name='output')(last_layer)
custom_vgg_model = Model(image_input, out)
# custom_vgg_model.summary()

for layer in custom_vgg_model.layers[:-1]:
    layer.trainable = False

custom_vgg_model.layers[3].trainable
# custom_vgg_model.summary()
 ###########################################################################################
#COMPILE
custom_vgg_model.compile(loss='binary_crossentropy',optimizer='rmsprop', metrics=['accuracy'])


# In[20]:


##############################################################################################
t=time.time()
#	t = now()
hist = custom_vgg_model.fit(X_train, y_train, batch_size=32, epochs=15, verbose=1, validation_data=(X_val, y_val))
print('Training time: %s' % (time.time() - t))


################################################################################################


# In[ ]:



##########################################################################

from sklearn.metrics import classification_report,confusion_matrix
import itertools
#################################################################
Y_pred = custom_vgg_model.predict(X_test) # NO OF IMAGES x NUMBER OF CLASSES
y_pred = np.round(Y_pred) # required to bring to proper format 1xcol


#%%
print(classification_report(y_test, y_pred)) #

# y = r"D:\data-nyco\images-resize-240-320\Number\train\0"  
# n = r'D:\data-nyco\images-resize-240-320\Number\test\0'


custom_vgg_model.save('D:\data-nyco\models\model.h5')
# model_json = model.to_json()
# with open("D:\data-nyco\models\model.json", "w") as json_file:
#     json_file.write(model_json)
# from keras.models import load_model

# new=load_model(r'D:\data-nyco\models\VGG16_QLL_311019.h5')
# new.summary()
# #  LOADING WEIGHTS


# # In[ ]:


# print(y_pred)



Loaded the images of dataset-0

Loaded the images of dataset-1

(9369, 224, 224, 3) (9369,) (728, 224, 224, 3) (728,)
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
574717952/574710816 [==============================] - 25s 0us/step
Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     738

NotImplementedError: numpy() is only available when eager execution is enabled.